# Checking the accuracy of a WCS

Malte Tewes, April 2024

A great intro to astropy SkyCoord and catalog matching: https://learn.astropy.org/tutorials/1-Coordinates-Intro

In [ ]:
import os
import numpy as np

from astropy import units as u
import astropy.table

%matplotlib ipympl
import matplotlib
from matplotlib import pyplot as plt

import ccdproc

from astropy import visualization as aviz
from astropy.nddata.blocks import block_reduce

from photutils.background import Background2D, MedianBackground

from astropy.convolution import convolve
from photutils.segmentation import make_2dgaussian_kernel

from photutils.segmentation import SourceFinder, SourceCatalog

from astropy.coordinates import SkyCoord

import pyvo

# Reading in the image

The header written by ASTAP has an error and needs a fix:

In [ ]:

filepath = "/Users/mtewes/Desktop/no_backup/2024-03-08/LIGHT/2024-03-08_23-53-42_i_-10.00_60.00s_0005.fits"

tmp_filepath = "/Users/mtewes/Desktop/no_backup/SIP-fix.fits"

# Fix the CIP header:

hdul = astropy.io.fits.open(filepath)
hdul.info()
hdul[0].header["CTYPE1"] = "RA---TAN-SIP"
hdul[0].header["CTYPE2"] = "DEC--TAN-SIP"

hdul.writeto(tmp_filepath, overwrite=True)
hdul.close()

img = ccdproc.CCDData.read(tmp_filepath, unit="adu")

# Use this to ignore SIP:
# wcsobj.sip = None

# Get GAIA data

In [ ]:
pos_string = "{} {}".format(img.header["OBJCTRA"], img.header["OBJCTDEC"])

c = SkyCoord(pos_string, unit=(u.hourangle, u.deg))
print(c)

In [ ]:
c_dec_deg = c.dec.to_value(u.deg)
c_ra_deg = c.ra.to_value(u.deg)
r_deg = 1.0

tap_service = pyvo.dal.TAPService("https://gaia.ari.uni-heidelberg.de/tap")


tap_query = """
SELECT ra, dec, phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag
FROM gaiadr3.gaia_source
WHERE 1 = CONTAINS(POINT('ICRS', ra, dec),
                   CIRCLE('ICRS', {}, {}, {}))
AND phot_g_mean_mag < 18.0
""".format(c_ra_deg, c_dec_deg, r_deg)
print(tap_query)

tap_data = tap_service.run_sync(tap_query)

#with warnings.catch_warnings():
#    warnings.filterwarnings("ignore", module='astropy.*')
#    tap_daten = tap_service.run_sync(tap_anfrage)

gaiacat = tap_data.to_table()

# We change the names of ra and dec, to avoid confusion
gaiacat.rename_columns(names=["ra", "dec"], new_names=["gaia_ra", "gaia_dec"])

print(gaiacat.info)

# Building the source catalog of our own file

In [ ]:
from astropy import visualization as aviz
from astropy.nddata.blocks import block_reduce
from astropy.nddata.utils import Cutout2D
from matplotlib import pyplot as plt


def show_image(image,
               percl=99, percu=None, is_mask=False,
               figsize=(10, 10),
               cmap='viridis', log=False, clip=True,
               show_colorbar=True, show_ticks=True,
               fig=None, ax=None, input_ratio=None):
    """
    Show an image in matplotlib with some basic astronomically-appropriat stretching.

    Parameters
    ----------
    image
        The image to show
    percl : number
        The percentile for the lower edge of the stretch (or both edges if ``percu`` is None)
    percu : number or None
        The percentile for the upper edge of the stretch (or None to use ``percl`` for both)
    figsize : 2-tuple
        The size of the matplotlib figure in inches
    """
    if percu is None:
        percu = percl
        percl = 100 - percl

    if (fig is None and ax is not None) or (fig is not None and ax is None):
        raise ValueError('Must provide both "fig" and "ax" '
                         'if you provide one of them')
    elif fig is None and ax is None:
        if figsize is not None:
            # Rescale the fig size to match the image dimensions, roughly
            image_aspect_ratio = image.shape[0] / image.shape[1]
            figsize = (max(figsize) * image_aspect_ratio, max(figsize))

        fig, ax = plt.subplots(1, 1, figsize=figsize)

    
    # To preserve details we should *really* downsample correctly and
    # not rely on matplotlib to do it correctly for us (it won't).

    # So, calculate the size of the figure in pixels, block_reduce to
    # roughly that,and display the block reduced image.

    # Thanks, https://stackoverflow.com/questions/29702424/how-to-get-matplotlib-figure-size
    fig_size_pix = fig.get_size_inches() * fig.dpi

    ratio = (image.shape // fig_size_pix).max()

    if ratio < 1:
        ratio = 1

    ratio = input_ratio or ratio

    reduced_data = block_reduce(image, ratio)

    if not is_mask:
        # Divide by the square of the ratio to keep the flux the same in the
        # reduced image. We do *not* want to do this for images which are
        # masks, since their values should be zero or one.
         reduced_data = reduced_data / ratio**2

    # Of course, now that we have downsampled, the axis limits are changed to
    # match the smaller image size. Setting the extent will do the trick to
    # change the axis display back to showing the actual extent of the image.
    extent = [0, image.shape[1], 0, image.shape[0]]

    if log:
        stretch = aviz.LogStretch()
    else:
        stretch = aviz.LinearStretch()

    norm = aviz.ImageNormalize(reduced_data,
                               interval=aviz.AsymmetricPercentileInterval(percl, percu),
                               stretch=stretch, clip=clip)

    if is_mask:
        # The image is a mask in which pixels should be zero or one.
        # block_reduce may have changed some of the values, so reset here.
        reduced_data = reduced_data > 0
        # Set the image scale limits appropriately.
        scale_args = dict(vmin=0, vmax=1)
    else:
        scale_args = dict(norm=norm)

    im = ax.imshow(reduced_data, origin='lower',
                   cmap=cmap, extent=extent, aspect='equal', **scale_args)

    if show_colorbar:
        # I haven't a clue why the fraction and pad arguments below work to make
        # the colorbar the same height as the image, but they do....unless the image
        # is wider than it is tall. Sticking with this for now anyway...
        # Thanks: https://stackoverflow.com/a/26720422/3486425
        #fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        #fig.colorbar(im, ax=ax)
        cax = fig.add_axes([ax.get_position().x1+0.01,ax.get_position().y0,0.02,ax.get_position().height])
        plt.colorbar(im, cax=cax)
        # In case someone in the future wants to improve this:
        # https://joseph-long.com/writing/colorbars/
        # https://stackoverflow.com/a/33505522/3486425
        # https://matplotlib.org/mpl_toolkits/axes_grid/users/overview.html#colorbar-whose-height-or-width-in-sync-with-the-master-axes

    if not show_ticks:
        ax.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False)



In [ ]:
show_image(img, log=False, percl=20, percu=99.9)

In [ ]:
# Removing the background

data = img.data + 0.0

bkg_estimator = MedianBackground()
bkg = Background2D(data, (100, 100), filter_size=(3, 3), bkg_estimator=bkg_estimator)

bkg.background
show_image(ccdproc.CCDData(bkg.background, unit="adu"))

data -= bkg.background  # subtract the background

threshold = 3.0 * bkg.background_rms # this is an array


In [ ]:
# Filtering (convolving) for source detection

kernel = make_2dgaussian_kernel(2.0, size=5)  # FWHM = 2.0
convolved_data = convolve(data, kernel)
#show_image(ccdproc.CCDData(convolved_data, unit="adu"))

In [ ]:
# Segmenting the image

finder = SourceFinder(npixels=4, connectivity=4, progress_bar=False)
#segment_map = finder(convolved_data, threshold)
segment_map = finder(data, threshold)
print(segment_map)

# And measuring sources
cat = SourceCatalog(data, segment_map, convolved_data=convolved_data, wcs=img.wcs)
tbl = cat.to_table(columns=["xcentroid_win", "ycentroid_win", "sky_centroid_win", "fwhm", "max_value", "kron_flux", "segment_flux"])
tbl

In [ ]:
# Plotting mag and size

tbl["mag"] = -2.5 * np.log10(tbl["kron_flux"].value) + 20

#plt.scatter(tbl["semiminor_sigma"], tbl["kron_flux"])
#plt.hist2d(tbl["fwhm"].value, tbl["mag"], bins=20, range=[[1.0, 5.0], [7, 13.0]])
plt.figure()
plt.plot(tbl["fwhm"].value, tbl["mag"], "r.", markersize=1)
plt.xlim(0, 15)
plt.gca().invert_yaxis()
plt.show()

# Matching the catalogs

In [ ]:
gaiacoords = SkyCoord(ra=gaiacat['gaia_ra'], dec=gaiacat['gaia_dec'])

idx_gaia, sep2d_gaia, _ = gaiacoords.match_to_catalog_sky(tbl["sky_centroid_win"])
good_ids = sep2d_gaia < 2.0 * u.arcsec

print(len(tbl))
print(len(idx_gaia))

fullcat = astropy.table.hstack([tbl[idx_gaia][good_ids], gaiacat[good_ids]])
print(len(fullcat))

fullcat["error_ra_deg"] = fullcat["sky_centroid_win"].ra.to_value(u.deg) - fullcat["gaia_ra"].value
fullcat["error_dec_deg"] = fullcat["sky_centroid_win"].dec.to_value(u.deg) - fullcat["gaia_dec"].value
fullcat

In [ ]:
plt.figure()
plt.hist(sep2d_gaia.arcsec, histtype='step', 
         bins=np.logspace(-2, 2., 64))
plt.xlabel('separation [arcsec]')
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
plt.figure()
plt.hist(np.array(fullcat["phot_g_mean_mag"].value), bins=20)
plt.xlabel('phot_g_mean_mag')
plt.show()


In [ ]:
fig1, ax1 = plt.subplots()

Q = ax1.quiver(fullcat["gaia_ra"], fullcat["gaia_dec"], fullcat["error_ra_deg"]*3600.0, fullcat["error_dec_deg"]*3600.0, units='dots', scale=0.03)
#Q = ax1.quiver(fullcat["xcentroid_win"], fullcat["ycentroid_win"], fullcat["error_ra_deg"]*3600.0, fullcat["error_dec_deg"]*3600.0, units='dots', scale=0.03)

#ax1.plot(fullcat["sky_centroid_win"].ra.deg, fullcat["sky_centroid_win"].dec.deg, "r.", markersize=1)
qk = ax1.quiverkey(Q, 0.9, 0.9, 1, "1''", labelpos='E', coordinates='figure')
ax1.set_xlabel("RA [deg]")
ax1.set_ylabel("Dec [deg]")

In [ ]:
# attempt to refit the WCS

sky = SkyCoord(ra=fullcat['gaia_ra'], dec=fullcat['gaia_dec'])
print(len(sky))
print((np.array(fullcat["xcentroid_win"]), np.array(fullcat["ycentroid_win"])))

newwcs = astropy.wcs.utils.fit_wcs_from_points(
    (np.array(fullcat["xcentroid_win"]), np.array(fullcat["ycentroid_win"])),
    SkyCoord(ra=fullcat['gaia_ra'], dec=fullcat['gaia_dec']),
    proj_point='center', projection='TAN', sip_degree=None)